<a href="https://colab.research.google.com/github/shivammehta007/NLPResearch/blob/master/Tweet2EmotionVectors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tweet To Emotion
#### Converting User input tweets to emotion prediction

In [37]:
import random
import os
import re

import numpy as np
import pandas as pd

import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F


from torchtext import data
from torchtext.data import Field
from torchtext.data import TabularDataset


from string import punctuation

import nltk
from nltk import word_tokenize
import spacy

import tqdm

nltk.download('punkt')
nlp = spacy.load('en')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
# Setting up Custom SEED
SEED = 1234
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
np.random.seed(SEED)
torch.backends.cudnn.deterministic = True
random.seed(SEED)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# Change if location is different
data_location = 'drive/My Drive/Homeworks/SNARecSysProject/'

In [13]:
os.listdir(data_location)

['SemVal-Dataset']

## Data Loading

### Helper Functions

#### Reading Filenames

In [0]:
def filenames(current_dataset_name, train='train.csv', val='val.csv', test='test.csv'):
    """
    Loads the current dataset into memory and returns its dataframes
    Input Parameters:
    current_dataset_name -> string: Name of folder containing this dataset
    """
    train = os.path.join(data_location, current_dataset_name, train), 
    val = os.path.join(data_location, current_dataset_name, val) if val and os.path.exists(os.path.join(data_location, current_dataset_name, val)) else None
    test = os.path.join(data_location, current_dataset_name, test) if test and os.path.exists(os.path.join(data_location, current_dataset_name, test)) else None
    return train, val, test

### Current DataSet: SemVal

#### Creating Filenames

In [0]:
current_dataset_name = 'SemVal-Dataset'
filename_semval_train, filename_semval_val, filename_semval_test = filenames(current_dataset_name)

#### Creating DataLoader

In [0]:
TEXT = Field(sequential=True, tokenize='spacy', lower=True)
LABEL = Field(sequential=False, use_vocab=False, dtype=torch.long)

In [0]:
semval_datafields = [(None, None), ('ID', None), ('Tweet', TEXT),
                     ('anger', LABEL), ('anticipation', LABEL),
                     ('disgust', LABEL), ('fear', LABEL),
                     ('joy', LABEL), ('love', LABEL),
                     ('optimism', LABEL), ('pessimism', LABEL),
                     ('sadness', LABEL), ('surprise', LABEL)]
trn, vld = TabularDataset.splits(
               path="data", # the root directory where the data lies
               train='train.csv', validation="valid.csv",
               format='csv',
               skip_header=True, # if your csv header has a header, make sure to pass this to ensure it doesn't get proceesed as data!
               fields=tv_datafields)

tst_datafields = [("id", None), # we won't be needing the id, so we pass in None as the field
                  ("comment_text", TEXT)]
tst = TabularDataset(
           path="data/test.csv", # the file path
           format='csv',
           skip_header=True, # if your csv header has a header, make sure to pass this to ensure it doesn't get proceesed as data!
           fields=tst_datafields)